# LIME Explainability for FinBERT

This notebook demonstrates LIME (Local Interpretable Model-agnostic Explanations) integration for explaining FinBERT sentiment predictions.

**FYP-159: Integrate LIME for sample analysis**

## Setup

In [ ]:
import sys
from pathlib import Path

# Add backend to path
sys.path.insert(0, str(Path.cwd().parent / "backend"))

from app.explainability import LIMEExplainer, get_lime_explainer
from app.explainability.visualizations import (
    plot_lime_features,
    plot_lime_summary_bar,
    plot_lime_class_comparison,
    save_lime_html,
)

import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')

## Initialize LIME Explainer

In [ ]:
# Initialize LIME explainer (singleton pattern)
explainer = get_lime_explainer(num_features=10, num_samples=1000)
print("LIME explainer initialized successfully!")

## Example 1: Positive Sentiment

In [ ]:
text_positive = "Stock prices surged to record highs as investors celebrated strong earnings"

# Generate explanation
explanation_pos = explainer.explain(text_positive, num_features=10)

# Display prediction
pred = explanation_pos["prediction"]
print(f"Text: {text_positive}")
print(f"Prediction: {pred['label'].upper()} ({pred['score']:.1%} confidence)")
print(f"\nTop features:")
for feature, weight in explanation_pos["top_features"][:5]:
    print(f"  {feature}: {weight:+.4f}")

In [ ]:
# Visualize
plot_lime_features(explanation_pos, figsize=(12, 6))

## Example 2: Negative Sentiment

In [ ]:
text_negative = "Markets crashed following disappointing employment figures and weak economic indicators"

# Generate explanation
explanation_neg = explainer.explain(text_negative, num_features=10)

# Display prediction
pred = explanation_neg["prediction"]
print(f"Text: {text_negative}")
print(f"Prediction: {pred['label'].upper()} ({pred['score']:.1%} confidence)")
print(f"\nTop features:")
for feature, weight in explanation_neg["top_features"][:5]:
    print(f"  {feature}: {weight:+.4f}")

In [ ]:
# Visualize
plot_lime_features(explanation_neg, figsize=(12, 6))

## Example 3: Neutral Sentiment

In [ ]:
text_neutral = "The Federal Reserve announced its decision to maintain current interest rates"

# Generate explanation
explanation_neu = explainer.explain(text_neutral, num_features=10)

# Display prediction
pred = explanation_neu["prediction"]
print(f"Text: {text_neutral}")
print(f"Prediction: {pred['label'].upper()} ({pred['score']:.1%} confidence)")
print(f"\nTop features:")
for feature, weight in explanation_neu["top_features"][:5]:
    print(f"  {feature}: {weight:+.4f}")

In [ ]:
# Visualize
plot_lime_features(explanation_neu, figsize=(12, 6))

## Batch Analysis: Multiple Texts

In [ ]:
# Define multiple texts for batch analysis
texts = [
    "Stock prices surged after earnings beat",
    "Market crashed on recession fears",
    "Investors cautious about economic outlook",
    "Revenue growth exceeded expectations",
    "Company announced major layoffs",
]

# Generate explanations
explanations = explainer.explain_batch(texts, num_features=10)

# Display results
for i, (text, exp) in enumerate(zip(texts, explanations), 1):
    if exp:
        pred = exp["prediction"]
        print(f"{i}. [{pred['label'].upper():8s}] {pred['score']:.1%} - {text}")
    else:
        print(f"{i}. [FAILED] - {text}")

## Summary Analysis

In [ ]:
# Get summary data
summary_data = explainer.get_summary_data(explanations, top_n=15)

print(f"Analyzed {summary_data['num_explanations']} texts")
print(f"\nTop 10 most important features overall:")
for feature, importance in summary_data['top_features'][:10]:
    print(f"  {feature}: {importance:.4f}")

In [ ]:
# Plot summary bar chart
plot_lime_summary_bar(summary_data, top_n=15, figsize=(14, 8))

In [ ]:
# Plot class comparison
plot_lime_class_comparison(summary_data, top_n=10, figsize=(16, 10))

## Export HTML Explanation

In [ ]:
# Save an explanation as interactive HTML
output_path = Path("../data/processed/explanations/lime_example_interactive.html")
save_lime_html(explanation_pos, output_path)
print(f"Saved interactive HTML to: {output_path}")

## LIME vs SHAP Comparison

Compare LIME and SHAP explanations for the same text.

In [ ]:
from app.explainability import get_explainer
from app.explainability.visualizations import plot_token_contributions

# Get SHAP explainer
shap_explainer = get_explainer()

# Same text for both
text = "Stock prices surged on positive earnings"

# Get explanations
lime_exp = explainer.explain(text, num_features=10)
shap_exp = shap_explainer.explain(text)

print("LIME Prediction:", lime_exp["prediction"]["label"].upper())
print("SHAP Prediction:", shap_exp["prediction"]["label"].upper())

In [ ]:
# Plot side-by-side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

# LIME plot
plot_lime_features(lime_exp, show=False)
plt.title("LIME Explanation")

# SHAP plot
plot_token_contributions(shap_exp, show=False)
plt.title("SHAP Explanation")

plt.tight_layout()
plt.show()

## Conclusion

This notebook demonstrated:
- ✅ LIME explainer initialization
- ✅ Single text explanations (positive, negative, neutral)
- ✅ Batch explanations
- ✅ Summary visualizations
- ✅ Feature importance analysis
- ✅ HTML export for interactive exploration
- ✅ Comparison with SHAP

**FYP-159 Completed**: LIME integration provides token-level sentiment reasoning with visualizations.